In [27]:
import json

# Read capital names and coordinates from json file
try:
  capitals_json = json.load(open('capitals.json'))
except:
  import urllib.request
  url = 'https://raw.githubusercontent.com/Gurobi/modeling-examples/master/traveling_salesman/capitals.json'
  data = urllib.request.urlopen(url).read()
  capitals_json = json.loads(data)

capitals = []
coordinates = {}
for state in capitals_json:
    if state not in ['AK', 'HI']:
      capital = capitals_json[state]['capital']
      capitals.append(capital)
      coordinates[capital] = (float(capitals_json[state]['lat']), float(capitals_json[state]['long']))

In [28]:
from geopy.distance import geodesic

# City coordinates
city_coordinates = coordinates

def calculate_distance(coord1, coord2):
    """Calculate the distance between two coordinates using geodesic distance."""
    return geodesic(coord1, coord2).miles

def nearest_neighbor(curr_city, unvisited_cities):
    """Find the nearest neighbor to the current city from the unvisited cities."""
    min_distance = float('inf')
    nearest_city = None

    for city in unvisited_cities:
        distance = calculate_distance(city_coordinates[curr_city], city_coordinates[city])
        if distance < min_distance:
            min_distance = distance
            nearest_city = city

    return nearest_city, min_distance

def tsp_nearest_neighbor(start_city):
    """Solve TSP using the nearest neighbor heuristic."""
    unvisited_cities = list(city_coordinates.keys())
    unvisited_cities.remove(start_city)
    tour = [start_city]
    total_distance = 0

    while unvisited_cities:
        nearest_city, min_distance = nearest_neighbor(tour[-1], unvisited_cities)
        tour.append(nearest_city)
        total_distance += min_distance
        unvisited_cities.remove(nearest_city)

    # Return to the starting city to complete the tour
    tour.append(start_city)
    total_distance += calculate_distance(city_coordinates[tour[-2]], city_coordinates[start_city])

    return tour, total_distance

# Choose a starting city (e.g., Montgomery)
starting_city = 'Montgomery'

# Solve TSP using nearest neighbor heuristic
tour, total_distance = tsp_nearest_neighbor(starting_city)

# Print the tour and total distance
print("Tour:", ' -> '.join(tour))
print("Total Distance:", total_distance, "miles")



Tour: Montgomery -> Atlanta -> Columbia -> Raleigh -> Richmond -> Annapolis -> Dover -> Trenton -> Harrisburg -> Albany -> Hartford -> Providence -> Boston -> Concord -> Montpelier -> Augusta -> Charleston -> Columbus -> Frankfort -> Indianapolis -> Springfield -> Jefferson City -> Topeka -> Lincoln -> Des Moines -> Saint Paul -> Madison -> Lansing -> Nashville -> Little Rock -> Jackson -> Baton Rouge -> Austin -> Oklahoma City -> Santa Fe -> Denver -> Cheyenne -> Pierre -> Bismarck -> Helana -> Boise -> Salt Lake City -> Carson City -> Sacramento -> Salem -> Olympia -> Phoenix -> Tallahassee -> Montgomery
Total Distance: 12851.320283175473 miles


In [31]:
%pip install folium

  Using cached folium-0.14.0-py2.py3-none-any.whl (102 kB)
  Using cached branca-0.6.0-py3-none-any.whl (24 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\krish\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [32]:
import folium

# Previous code for calculating distances and solving TSP

def plot_tsp_on_map(tour):
    """Plot the TSP tour on a geographical map."""
    tsp_map = folium.Map(location=[37.7749, -122.4194], zoom_start=4)  # Centered at the US

    # Plot cities
    for city in city_coordinates:
        folium.Marker(location=city_coordinates[city], popup=city).add_to(tsp_map)

    # Plot tour path
    for i in range(len(tour) - 1):
        start_coords = city_coordinates[tour[i]]
        end_coords = city_coordinates[tour[i + 1]]
        folium.PolyLine([start_coords, end_coords], color='blue').add_to(tsp_map)

    # Plot a line from the last city back to the starting city to complete the tour
    start_coords = city_coordinates[tour[-1]]
    end_coords = city_coordinates[tour[0]]
    folium.PolyLine([start_coords, end_coords], color='blue').add_to(tsp_map)

    return tsp_map

# Solve TSP using nearest neighbor heuristic
tour, total_distance = tsp_nearest_neighbor(starting_city)

# Plot TSP on map
tsp_map = plot_tsp_on_map(tour)

# Display the map
tsp_map.save('tsp_map.html')  # Save the map to an HTML file
tsp_map


In [33]:
def folium_deepnote_show(m):
    data = m.get_root().render()
    data_fixed_height = data.replace('width: 100%;height: 100%', 'width: 100%').replace('height: 100.0%;', 'height: 609px;', 1)
    display(HTML(data_fixed_height))